In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional     scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, e
             arly_stopping_rounds=50):
    if useTrainCV: #使用CV来调节n_estimator数目
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, 
                              label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds,metrics='auc', 
                          early_stopping_rounds=early_stopping_rounds, 
                          show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1] 
    feat_imp = pd.Series(alg.booster().get_fscore()
                        ).sort_values(ascending=False)

In [ ]:
# 估计n_estimators
xgb1 = XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,gamma=0,
 subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)

In [ ]:
param_test1 = {'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140,
            max_depth=5,min_child_weight=1, gamma=0, subsample=0.8,colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])

In [ ]:
param_test2 = {'max_depth':[4,5,6],'min_child_weight':[4,5,6]}
gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=140, 
        max_depth=5,min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])

In [ ]:
param_test2b = {'min_child_weight':[6,8,10,12]}
gsearch2b = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=140,
        max_depth=4,min_child_weight=2, gamma=0, subsample=0.8, 
    colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
    param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch2b.fit(train[predictors],train[target])
modelfit(gsearch3.best_estimator_, train, predictors)

In [ ]:
#优化gamma
param_test3 = {'gamma':[i/10.0 for i in range(0,5)]}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, 
max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, 
            objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])

In [ ]:
xgb2 = XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=4,min_child_weight=6,
 gamma=0,subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
scale_pos_weight=1,seed=27)
modelfit(xgb2, train, predictors)

In [ ]:
param_test4 = {'subsample':[i/10.0 for i in range(6,10)],
               'colsample_bytree':[i/10.0 for i in range(6,10)]}

gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=177, 
        max_depth=3, min_child_weight=4, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
            objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27),
        param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch4.fit(train[predictors],train[target])

In [ ]:
param_test5 = {'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]}

gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=177, 
    max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, 
    objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
    param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch5.fit(train[predictors],train[target])

In [ ]:
param_test6 = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=177, 
        max_depth=4, min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8, 
        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
        param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(train[predictors],train[target])

In [ ]:
param_test7 = {'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}
gsearch7 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=177, 
        max_depth=4, min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
        param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(train[predictors],train[target])

In [ ]:
xgb3 = XGBClassifier(learning_rate =0.1,n_estimators=1000,max_depth=4,min_child_weight=6,gamma=0,
 subsample=0.8,colsample_bytree=0.8,reg_alpha=0.005,objective= 'binary:logistic',nthread=4,
 scale_pos_weight=1,seed=27)
modelfit(xgb3, train, predictors)

In [ ]:
xgb4 = XGBClassifier(learning_rate =0.01, n_estimators=5000, max_depth=4,min_child_weight=6,
 gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005,objective= 'binary:logistic',
nthread=4, scale_pos_weight=1, seed=27)
modelfit(xgb4, train, predictors)

In [1]:
import xgboost as xgb